# Env Setup

In [110]:
# Env Setup
import numpy as np
import cv2 as cv
from PIL import Image
import os
import epd_utils as epd
import cnt_white_edge as cwe
from collections import Counter
import dither

import tqdm
from numba import jit

In [111]:
# Preparation
path_wkspace = 'Exp/new/'
path_src_home = path_wkspace
path_out_home = path_wkspace + 'out/'
# path_out_home = '\\\\192.168.1.9\\myshare\\IT8951\\pic\\exp\\star\\'
path_out_home = '\\\\192.168.137.114\\myshare\\IT8951\\pic\\exp\\misc\\diet\\'


# img_A = cv.rotate(img_A, cv.ROTATE_90_CLOCKWISE)
# img_B = cv.rotate(img_B, cv.ROTATE_90_CLOCKWISE)
#
# epd.saveAsBlackWhiteImage(path_dst_A, img_A)
# epd.saveAsBlackWhiteImage(path_dst_B, img_B)

# Pair 1

In [282]:
src_type = '.jpg'
dst_type = '.bmp'
A = '1'
B = '2'
path_src_A = path_src_home + '1.jpg'
path_src_B = path_src_home + '2.jpg'
path_dst_A = path_out_home + '1.bmp'
path_dst_B = path_out_home + '2.bmp'
# print(path_src_A)
img_A = cv.imread(path_src_A, cv.IMREAD_GRAYSCALE)
img_B = cv.imread(path_src_B, cv.IMREAD_GRAYSCALE)

img_A_dither = dither.floyd_steinberg(img_A / 255) * 255
img_B_dither = dither.floyd_steinberg(img_B / 255) * 255
# img_A = cv.rotate(img_A, cv.ROTATE_90_CLOCKWISE)
# img_B = cv.rotate(img_B, cv.ROTATE_90_CLOCKWISE)
# epd.saveAsBlackWhiteImage(path_dst_A, img_A)
# epd.saveAsBlackWhiteImage(path_dst_B, img_B)

In [113]:
# floodfill return filled pts
def fill(data, start_coords, fill_value, border_value, orig_val_bound=(10, 10), connectivity=8):
    """
    Flood fill algorithm

    Parameters
    ----------
    data : (M, N) ndarray of uint8 type
        Image with flood to be filled. Modified inplace.
    start_coords : tuple
        Length-2 tuple of ints defining (row, col) start coordinates.
    fill_value : int
        Value the flooded area will take after the fill.
    border_value: int
        Value of the color to paint the borders of the filled area with.
    connectivity: 4 or 8
        Connectivity which we use for the flood fill algorithm (4-way or 8-way).

    Returns
    -------
    filled_data: ndarray
        The data with the filled area.
    borders: ndarray
        The borders of the filled area painted with border_value color.
    """
    assert connectivity in [4, 8]

    filled_data = data.copy()
    # print(filled_data.shape)
    xsize, ysize = filled_data.shape
    orig_value = filled_data[start_coords[0], start_coords[1]]
    # orig_value = 245
    stack = set(((start_coords[0], start_coords[1]),))
    # if fill_value == orig_value:
    #     raise ValueError("Filling region with same value already present is unsupported. Did you already fill this region?")

    border_points = []
    filled_pts = []
    lower_bound, upper_bound = orig_val_bound
    lower_bound = orig_value - lower_bound
    upper_bound = orig_value + upper_bound
    upper_bound = upper_bound if upper_bound < 255 else 255
    lower_bound = lower_bound if lower_bound > 0 else 0
    while stack:
        x, y = stack.pop()
        # cond_fill = (filled_data[x, y] == orig_value)
        cond_fill = (filled_data[x, y] < upper_bound) and (filled_data[x, y] > lower_bound)
        if cond_fill:
            filled_data[x, y] = fill_value
            filled_pts.append((y,x))
            if x > 0:
                stack.add((x - 1, y))
            if x < (xsize - 1):
                stack.add((x + 1, y))
            if y > 0:
                stack.add((x, y - 1))
            if y < (ysize - 1):
                stack.add((x, y + 1))

            if connectivity == 8:
                if x > 0 and y > 0:
                    stack.add((x - 1, y - 1))
                if x > 0 and y < (ysize - 1):
                    stack.add((x - 1, y + 1))
                if x < (xsize - 1) and y > 0:
                    stack.add((x + 1, y - 1))
                if x < (xsize - 1) and y < (ysize - 1):
                    stack.add((x + 1, y + 1))
        else:
            if filled_data[x, y] != fill_value:
                border_points.append([x, y])

    # Fill all image with white
    borders = filled_data.copy()
    borders.fill(255)

    # Paint borders
    for x, y in border_points:
        borders[x, y] = border_value

    return filled_data, borders, set(filled_pts)


In [114]:
# get windowed White Edge matrix
def getAvgWinForWhiteEdge(img, ghost_threshold=14, step=(6, 6)):
    mImg = np.copy(img)
    h, w = mImg.shape
    # print(h,w)
    ghost_pts = {}
    x_step, y_step = step
    img3 = epd.createEmptyGrayScaleImage((int(h / y_step), int(w / x_step)), gray=0)
    acc = np.array([0])
    # plt.figure().set_size_inches(60, 80)
    for y in range(0, h, y_step):
        for x in range(0, w, x_step):
            avg = cv.mean(mImg[y:y + y_step, x:x + x_step])
            if avg[0] < ghost_threshold:
                continue
            mImg[y:y + y_step, x:x + x_step] = avg[0]
            img3[int(y / y_step), int(x / x_step)] = avg[0]
            ghost_pts[(int(x / x_step),int(y / y_step))] = avg[0]
        pass
    return mImg, img3, ghost_pts

In [115]:
# Visualize points in image
def colorPoint(img,pts,color):
    for p in pts:
        x,y = p
        r,g,b = color
        img[y,x,0] = r
        img[y,x,1] = g
        img[y,x,2] = b

def colorPoint2(img,pts,color):
    for p in pts:
        y,x = p
        r,g,b = color
        img[y,x,0] = r
        img[y,x,1] = g
        img[y,x,2] = b

def vizPtsInArea(shape,pts_list,color_list=None):
    canvas = np.zeros(shape=shape)
    canvas = cv.cvtColor(canvas.astype('uint8'), cv.COLOR_GRAY2RGB)
    canvas[:, :, :] = 255
    if color_list is None:
        color_list = [(0x66, 0x66, 0x99), (0xFF, 0xCC, 0), (0x99, 0xcc, 0x33), (0x99, 0x33, 0x66),
                      (0, 0x99, 0x66),(0x22, 0xFF, 0x66) , (0x22, 0x77, 0x66),(0x22, 0, 0x66),
                      (0xdd, 0xcc, 0xaa), (0, 0xff, 0xff), (0, 0xff, 0), (0x33, 0xff, 0xf0),
                      (0, 0, 0xff), (0xff, 0xff, 0), (0xff, 0, 0), (0, 0x33, 0x66), (0xff, 0x33, 0x66), (0xff, 0x33, 0xFF)]
    for i,pts in enumerate(pts_list):
        colorPoint(canvas,pts,color_list[i%len(color_list)])
    epd.showCv2Img(canvas)

## Get Downscaled Image B

In [116]:
# downscale img B
img_B_fullscale, img_B_downscale = epd.getAvgWin(img_B_dither)
# epd.showCv2Img(img_B_downscale)

## White Edge Ghost

### Get white edge points

In [117]:
# downscale White Egde Layer
we_cnt_lut, we_pos_lut, we_mat012 = cwe.getWhiteEdgeCnt2(img_A_dither, img_B_dither)
we_mat255 = we_mat012 * 120
we_mat_fullscale, we_mat_downscale, we_ds_pts = getAvgWinForWhiteEdge(we_mat255, ghost_threshold=14)

#### Visualize White Edge points

In [ ]:
# vizPtsInArea(we_mat012.shape, [list(we_cnt_lut.keys())])
# vizPtsInArea(we_mat_downscale.shape,[we_ds_pts])

KeyboardInterrupt: 

#### Floodfill for same colored area

In [118]:
import time

# Floodfill for same colored area
h, w = we_mat_downscale.shape
ibd = np.copy(img_B_downscale)

we_ds_pt_set = set(we_ds_pts)
orig_area_pt_lists = []
we_area_pt_lists = []
area_lists = []
cnt = 0
t0 = time.time()
while len(we_ds_pt_set) > 0:
    p = we_ds_pt_set.pop()
    x1,y = p
    _, _, fpts = fill(ibd, (y, x1), 255, 0, orig_val_bound=(15, 15))
    if len(fpts) > 1:
        we_area_pts = we_ds_pt_set & fpts
        if (len(we_area_pts)<5):
            continue
        orig_area_pt_lists.append(fpts)
        we_area_pt_lists.append(we_area_pts)
        area_lists.append((fpts,we_area_pts))
    we_ds_pt_set = we_ds_pt_set - set(fpts)
print(time.time()-t0)

0.0600125789642334


In [93]:
# vizPtsInArea(we_mat_downscale.shape, orig_area_pt_lists[2:3])

#### Diet1 Make region whiter

In [164]:
#### Diet1 Make region whiter
area = orig_area_pt_lists[2]
delta = list(set(area)-set(we_ds_pts))
vizPtsInArea(we_mat_downscale.shape, [delta])

In [165]:
#### Diet1
def getMatFromPts(img, ds_pts):
    new_img = img.copy()
    for p in ds_pts:
        x,y = p
        new_img[y*6:(y+1)*6-1,x*6:(x+1)*6-1] = 1
    return new_img

In [166]:

rest_mat = np.zeros_like(img_B)
rest_mat = getMatFromPts(rest_mat,area)
epd.showCv2Img(rest_mat*240)


### Visualize White Minus Ghost

In [646]:

def getWhiteMinusCnt(img_A, img_B):
    A = np.copy(img_A).astype('int16') / 255
    B = np.copy(img_B).astype('int16') / 255
    D = B - A
    D = D + 1
    D = D / 2
    h, w = A.shape
    wm_pts = []
    for y in range(h):
        for x in range(w):
            if D[y, x] == 1:
                wm_pts.append((x, y))
    return wm_pts

img_A = cv.imread(path_src_A, cv.IMREAD_GRAYSCALE)
img_B = cv.imread(path_src_B, cv.IMREAD_GRAYSCALE)

# img_A = epd.threshold(img_A, 238, 238, mode=3)
# img_B = epd.threshold(img_A, 238, 238, mode=3)

img_A_dither = dither.floyd_steinberg(img_A / 255) * 255
img_B_dither = dither.floyd_steinberg(img_B / 255) * 255
wm_pts = getWhiteMinusCnt(img_A_dither, img_B_dither)
# vizPtsInArea(we_mat012.shape, [wm_pts])

### Make white gray

#### Downscale

In [647]:
def getAvgWinForWhiteMinus(wm_pts, threshold=18):
    h = int(1872/6)
    w = int(1404/6)
    wm_ds_mat = np.zeros((h,w))
    wm_ds_pts = []
    for p in wm_pts:
        x,y = p
        x1 = int(x/6)
        y1 = int(y/6)
        wm_ds_mat[y1,x1] = wm_ds_mat[y1,x1] + 1
    for y in range(h):
        for x in range(w):
            if wm_ds_mat[y,x] > threshold:
                wm_ds_pts.append((x,y))
    return wm_ds_pts,wm_ds_mat

In [648]:
wm_ds_pts,wm_ds_mat = getAvgWinForWhiteMinus(wm_pts,threshold=6)

In [649]:
# vizPtsInArea(we_mat_downscale.shape, [wm_ds_pts])

#### Find Same Color Areas

In [650]:
we_ds_pt_set2 = set(wm_ds_pts)
x,y = we_ds_pt_set2.pop()
print(ibd[y,x])


255


#### Find White Color Area

In [651]:

wpts = []
h,w = img_B_Kill_WE.shape
for y in range(h):
    for x in range(w):
        if img_B_Kill_WE[y,x] > 238:
            wpts.append((x,y))

WhitePts = wpts
# vizPtsInArea(we_mat_fullscale.shape, [wpts])



In [652]:
WhitePts2 = []
h,w = img_B_Kill_WE.shape
img_d = dither.floyd_steinberg(img_B_Kill_WE/255)*255
for y in range(h):
    for x in range(w):
        if img_B_Kill_WE[y,x] == 255:
            WhitePts2.append((x,y))
# vizPtsInArea(we_mat_fullscale.shape, [WhitePts2])

In [653]:
WhitePtsExceptWM = list(set(WhitePts) - set(wm_pts))
WhitePtsExceptWM2 = list(set(WhitePts2) - set(wm_pts))
vizPtsInArea(we_mat_fullscale.shape, [WhitePtsExceptWM2])


In [654]:
# wm_part0 =
wm_part0 = [] # body area
for p in WhitePts[251500:848900]:
    x,y = p
    if x <100 or x > 1300 or y < 136*6:
        continue
    wm_part0.append(p)
wm_part1 = list(set(wm_part0)-set(wm_pts)) # for no ghost
wm_part01 = list(set(wm_part0)&set(wm_pts))

# vizPtsInArea(we_mat_fullscale.shape, [wm_part01])


In [655]:

wm_pts2 = []
for p in wm_pts:
    x,y = p
    if y < 136*6:
        continue
    wm_pts2.append(p)
wm_part2 = list(set(wm_part0)-set(wm_pts2))
# vizPtsInArea(we_mat_fullscale.shape, [wm_part2])

#### Make White Minus Area Grayer

In [656]:
part2_mat = cv.imread('part21.bmp', cv.IMREAD_GRAYSCALE) # Collar area
# epd.showCv2Img(part2_mat)
part2_mat = part2_mat.astype('int16') / 255

In [657]:
def makeAreaGrayer(img, pts, val):
    new_img =  np.copy(img)
    for p in tqdm.tqdm(pts):
        x,y = p
        tmp = new_img[y,x] - val
        new_img[y,x] = tmp if tmp > 0 else 0
    return new_img

In [658]:
print(np.unique(part2_mat))

[0. 1.]


In [659]:
# vizPtsInArea(we_mat_fullscale.shape, [wm_part01])

In [660]:
collar_hair_mat = cv.imread('collar_hair.bmp', cv.IMREAD_GRAYSCALE) # Collar head area
# epd.showCv2Img(collar_head_mat)
# print(np.unique(collar_head_mat))
collar_hair_mat = collar_hair_mat.astype('int16') / 255
print(np.unique(collar_hair_mat))

[0. 1.]


In [664]:
WM2GRAY = np.copy(img_B_Kill_WE)
# part1[100:300,100:300] = 255

# wm2gray = makeAreaGrayer(WM2GRAY, wm_part0, 1) # gray body
wm2gray = makeAreaGrayer(WM2GRAY, wm_part1, 19)#17 # gray face
# wm2gray = makeAreaGrayer(wm2gray, wm_part01, 5)#12) # at collar, gray heavily hair, lightly cloth
wm2gray = wm2gray - collar_hair_mat*12#9 # gary collar hair
wm2gray = wm2gray - part2_mat*5 # gray collar
wm2gray = dither.floyd_steinberg(wm2gray / 255) * 255
wm2gray = cv.rotate(wm2gray, cv.ROTATE_90_CLOCKWISE)
epd.saveAsBlackWhiteImage(path_dst_B1, wm2gray)
print('Done')

100%|██████████| 305054/305054 [00:00<00:00, 487977.88it/s]


Done


## White Minus Ghost Specially

# Make BD'-1

Bd'-1: 头发部分的残影，全屏黑色颜色变浅，衣服部分，不够白和白的部分都加深

In [507]:
img_A = cv.imread(path_src_A, cv.IMREAD_GRAYSCALE)
img_B = cv.imread(path_src_B, cv.IMREAD_GRAYSCALE)

img_A_dither = dither.floyd_steinberg(img_A / 255) * 255
img_B_dither = dither.floyd_steinberg(img_B / 255) * 255

path_dst_BD1 = path_out_home + 'BD1' + dst_type

## Whiter Black

In [114]:
black_mat = img_B.copy()
h,w = black_mat.shape
for y in range(h):
    for x in range(w):
        if black_mat[y,x] > 30:
            black_mat[y,x] = 0
        else:
            black_mat[y,x] = 1
print(np.unique(black_mat))

[0 1]


In [202]:
# epd.showCv2Img(black_mat*255)

In [509]:
factor = 95#110
blackWhiter = img_B.copy() + black_mat*factor
AfterBlackWhiter = blackWhiter.copy()
blackWhiter = dither.floyd_steinberg(blackWhiter/255)*255
blackWhiter = cv.rotate(blackWhiter, cv.ROTATE_90_CLOCKWISE)
epd.saveAsBlackWhiteImage(path_dst_BD1, blackWhiter)
print('Done')

Done


## Darker White

In [510]:
white_mat = img_B.copy()
h,w = black_mat.shape
for y in range(h):
    for x in range(w):
        if white_mat[y,x] > 245:
            white_mat[y,x] = 1
        else:
            white_mat[y,x] = 0
print(np.unique(white_mat))

[0 1]


In [240]:
# epd.showCv2Img(white_mat*255)

In [518]:
factor = 95#95
whiteDarker = AfterBlackWhiter.copy() - white_mat*factor
whiteDarker = dither.floyd_steinberg(whiteDarker/255)*255
whiteDarker = cv.rotate(whiteDarker, cv.ROTATE_90_CLOCKWISE)
epd.saveAsBlackWhiteImage(path_dst_BD1, whiteDarker)
print('Done')

Done


# Make BD'-1-1

In [115]:
img_A = cv.imread(path_src_A, cv.IMREAD_GRAYSCALE)
img_B = cv.imread(path_src_B, cv.IMREAD_GRAYSCALE)

img_A_dither = dither.floyd_steinberg(img_A / 255) * 255
img_B_dither = dither.floyd_steinberg(img_B / 255) * 255
path_out_home = '\\\\192.168.137.114\\myshare\\IT8951\\pic\\exp\\opera\\'
path_dst_BD11 = path_out_home + 'E' + dst_type

## Whiter Black

In [116]:
black_mat3 = img_B.copy()
h, w = black_mat3.shape
for y in range(h):
    for x in range(w):
        if black_mat3[y, x] > 30:
            black_mat3[y, x] = 0
        else:
            black_mat3[y, x] = 1
print(np.unique(black_mat3))

[0 1]


In [117]:
# epd.showCv2Img(black_mat3*255)

In [118]:
factor = 95#110
blackWhiter = img_B.copy() + black_mat*factor
AfterBlackWhiter3 = blackWhiter.copy()
blackWhiter = dither.floyd_steinberg(blackWhiter/255)*255
blackWhiter = cv.rotate(blackWhiter, cv.ROTATE_90_CLOCKWISE)
epd.saveAsBlackWhiteImage(path_dst_BD11, blackWhiter)
print('Done')

Done


## Darker White Minus specially

In [119]:
def getWhiteMinusCnt2(img_A, img_B):
    A = np.copy(img_A).astype('int16') / 255
    B = np.copy(img_B).astype('int16') / 255
    D = B - A
    D = D + 1
    D = D / 2
    h, w = A.shape
    wm_pts = []
    for y in range(h):
        for x in range(w):
            if D[y, x] == 1:
                wm_pts.append((x, y))
    return wm_pts, D.astype('int16')

In [120]:
wm_pts,wm_mat = getWhiteMinusCnt2(img_A_dither, img_B_dither)

In [121]:
white_mat3 = img_B.copy()
h,w = white_mat3.shape
white_pts2 = []
for y in range(h):
    for x in range(w):
        if white_mat3[y,x] > 238:
            white_mat3[y,x] = 1
            white_pts2.append((x,y))
        else:
            white_mat3[y,x] = 0
print(np.unique(white_mat3))

[0 1]


In [122]:
non_wm_white_pts = list(set(white_pts2)-set(wm_pts))

In [123]:
# vizPtsInArea(img_B.shape, [white_pts2,non_wm_white_pts])

In [124]:
def isInImage2(x,y,h,w):
    return (x<w and x >= 0 and y < h and y >= 0)
def getDarkerMatAndPts11(wmmat, nonwm_white_pts, threshold=4):
    dark_mat = np.zeros_like(wmmat)
    dark_pts = []
    h,w = wmmat.shape
    a = []
    for p in nonwm_white_pts:
        x,y = p
        adjacent_cnt = 0
        for i in [x,x-1,x+1]:
            for j in [y,y-1,y+1]:
                if i==x and j==y:
                    continue
                # print((i,j))
                if isInImage2(i,j,h,w):
                    adjacent_cnt = adjacent_cnt + 1
        a.append(adjacent_cnt)
        if adjacent_cnt < threshold:
            dark_mat[y,x] = 1
            dark_pts.append((x,y))
        # print(Counter(a))
    return dark_mat,dark_pts

def getDarkerMatAndPts12(wmmat, nonwm_white_pts, threshold=4):
    dark_mat = np.zeros_like(wmmat)
    dark_pts = []
    h,w = wmmat.shape
    a = []
    for p in nonwm_white_pts:
        x,y = p
        adjacent_cnt = 0
        for i in [x,x-2,x+2]:
            for j in [y,y-2,y+2]:
                if i==x and j==y:
                    continue
                if isInImage(i,j,h,w) and wmmat[j,i] == 1:
                    adjacent_cnt = adjacent_cnt + 1
        # a.append(adjacent_cnt)
        if adjacent_cnt < threshold:
            dark_mat[y,x] = 1
            dark_pts.append((x,y))
        # print(Counter(a))
    return dark_mat,dark_pts

In [125]:
darker_mat2, darker_pts = getDarkerMatAndPts12(wm_mat, non_wm_white_pts,threshold=1)

In [126]:
factor = 13#20
whiteMinusDarker2 = AfterBlackWhiter3.copy() - darker_mat2*factor
epd.saveAsGrayScaleImage(path_out_home+'BD-1-1G.jpg',whiteMinusDarker2)
whiteMinusDarker2 = dither.floyd_steinberg(whiteMinusDarker2/255)*255
whiteMinusDarker2 = cv.rotate(whiteMinusDarker2, cv.ROTATE_90_CLOCKWISE)
epd.saveAsBlackWhiteImage(path_dst_BD11, whiteMinusDarker2)
print('Done')

Done


# Make BD'-2

In [283]:
img_A = cv.imread(path_src_A, cv.IMREAD_GRAYSCALE)
img_B = cv.imread(path_src_B, cv.IMREAD_GRAYSCALE)

img_A_dither = dither.floyd_steinberg(img_A / 255) * 255
img_B_dither = dither.floyd_steinberg(img_B / 255) * 255

path_dst_BD2 = path_out_home + '1B2' + dst_type

## Darker White Edge

In [284]:
we_cnt_lut, we_pos_lut, we_mat012 = cwe.getWhiteEdgeCnt2(img_A_dither, img_B_dither)

In [285]:
# epd.showCv2Img(we_mat012*120)

In [286]:
factor = 25#110
whiteEdgeDarker = img_B.copy() - we_mat012*factor
AfterWhiteEdgeDarker = whiteEdgeDarker.copy()
whiteEdgeDarker = dither.floyd_steinberg(whiteEdgeDarker/255)*255
whiteEdgeDarker = cv.rotate(whiteEdgeDarker, cv.ROTATE_90_CLOCKWISE)
epd.saveAsBlackWhiteImage(path_dst_BD2, whiteEdgeDarker)
print('Done')

Done


## Darker White Edge specially

In [287]:
black_mat2 = img_B.copy()
h,w = black_mat2.shape
black_pts = []
for y in range(h):
    for x in range(w):
        if black_mat2[y,x] > 30:
            black_mat2[y,x] = 0
        else:
            black_mat2[y,x] = 1
            black_pts.append((x,y))
white_mat2 = img_B.copy()
white_pts2 = []
h,w = white_mat2.shape
for y in range(h):
    for x in range(w):
        if white_mat2[y,x] > 245:
            white_mat2[y,x] = 1
            white_pts2.append((x,y))
        else:
            white_mat2[y,x] = 0
print(np.unique(white_pts2))
print(np.unique(black_mat2))

[   0    1    2 ... 1869 1870 1871]
[0 1]


In [288]:
# epd.showCv2Img(black_mat2*200)

In [289]:
def isInImage(x,y,h,w):
    return (x<w and x >= 0 and y < h and y >= 0)
def getDarkerMatAndPts3(wmat, wepts, threshold=4):
    dark_mat = np.zeros_like(wmat)
    dark_pts = []
    h,w = wmat.shape
    a = []
    for p in wepts:
        x,y = p
        adjacent_cnt = 0
        for i in [x,x-1,x+1]:
            for j in [y,y-1,y+1]:
                if i==x and j==y:
                    continue
                # print((i,j))
                if isInImage(i,j,h,w) and wmat[j, i] == 1:
                    adjacent_cnt = adjacent_cnt + 1
        a.append(adjacent_cnt)
        if adjacent_cnt < threshold:
            dark_mat[y,x] = 1
            dark_pts.append((x,y))
        # print(Counter(a))
    return dark_mat,dark_pts

def getDarkerMatAndPts4(wmat, wepts, threshold=4):
    dark_mat = np.zeros_like(wmat)
    dark_pts = []
    h,w = wmat.shape
    a = []
    for p in wepts:
        x,y = p
        adjacent_cnt = 0
        for i in [x,x-2,x+2]:
            for j in [y,y-2,y+2]:
                if i==x and j==y:
                    continue
                if isInImage(i,j,h,w) and wmat[j, i] == 1:
                    adjacent_cnt = adjacent_cnt + 1
        # a.append(adjacent_cnt)
        if adjacent_cnt < threshold:
            dark_mat[y,x] = 1
            dark_pts.append((x,y))
        # print(Counter(a))
    return dark_mat,dark_pts

def getDarkerMatAndPts5(wmat, wepts, threshold=4):
    dark_mat = np.zeros_like(wmat)
    dark_pts = []
    h,w = wmat.shape
    a = []
    for p in wepts:
        x,y = p
        adjacent_cnt = 0
        for i in [x,x-5,x+5]:
            for j in [y,y-5,y+5]:
                if i==x and j==y:
                    continue
                if isInImage(i,j,h,w) and wmat[j, i] == 1:
                    adjacent_cnt = adjacent_cnt + 1
        # a.append(adjacent_cnt)
        if adjacent_cnt < threshold:
            dark_mat[y,x] = 1
            dark_pts.append((x,y))
        # print(Counter(a))
    return dark_mat,dark_pts

def getDarkerMatAndPts6(wmat, welut, threshold=4):
    dark_mat = np.zeros_like(wmat)
    dark_pts = []
    h,w = wmat.shape
    a = []
    wepts  = list(welut.keys())
    for p in wepts:
        x,y = p
        adjacent_cnt = 0
        for i in [x,x-1,x+1]:
            for j in [y,y-1,y+1]:
                if i==x and j==y:
                    continue
                # print((i,j))
                if isInImage(i,j,h,w) and wmat[j, i] == 1:
                    adjacent_cnt = adjacent_cnt + 1
        a.append(adjacent_cnt)
        if adjacent_cnt < threshold:
            dark_mat[y,x] = welut[(x,y)]
            dark_pts.append((x,y))
        # print(Counter(a))
    return dark_mat,dark_pts

def getDarkerMatAndPts8(wmat, wepts, threshold=4):
    dark_mat = np.zeros_like(wmat)
    dark_pts = []
    h,w = wmat.shape
    a = []
    for p in wepts:
        x,y = p
        adjacent_cnt = 0
        for i in [x,x-8,x+8]:
            for j in [y,y-8,y+8]:
                if i==x and j==y:
                    continue
                if isInImage(i,j,h,w) and wmat[j, i] == 1:
                    adjacent_cnt = adjacent_cnt + 1
        # a.append(adjacent_cnt)
        if adjacent_cnt < threshold:
            dark_mat[y,x] = 1
            dark_pts.append((x,y))
        # print(Counter(a))
    return dark_mat,dark_pts

In [290]:
darker_mat3, darker_pts2 = getDarkerMatAndPts8(white_mat2, list(we_cnt_lut.keys()),threshold=1)
# darker_mat3, darker_pts2 = getDarkerMatAndPts6(white_mat2, we_cnt_lut,threshold=1)
darker_mat3 = darker_mat3.astype('float64')

In [291]:
# epd.showCv2Img(we_mat012*125)

In [292]:
# epd.showCv2Img(darker_mat3*240)
# print(Counter(darker_mat3.flatten()))

# print(len(darker_pts2))

thr 6 Counter({0: 2565254, 1: 63034})
thr 1 Counter({0: 2565432, 1: 62856})

In [293]:
# print(np.unique((darker_mat3-we_mat012)))
# if (darker_mat3==we_mat012).all():
#     print(True)
#
# print(type(we_mat012))
# print(we_mat012.dtype)
# print(darker_mat3.dtype)
# darker_mat3 = darker_mat3.astype('float64')

In [333]:
factor = 28#95
whiteEdgeDarker2 = img_B.copy() - darker_mat3*factor
AfterWhiteEdgeDarker = whiteEdgeDarker2.copy()
whiteEdgeDarker2 = dither.floyd_steinberg(whiteEdgeDarker2/255)*255
whiteEdgeDarker2 = cv.rotate(whiteEdgeDarker2, cv.ROTATE_90_CLOCKWISE)
epd.saveAsBlackWhiteImage(path_dst_BD2, whiteEdgeDarker2)
print('Done')

Done


In [295]:
# epd.showCv2Img(darker_mat3*240)

## Darker White Minus

In [296]:
def getWhiteMinusCnt2(img_A, img_B):
    A = np.copy(img_A).astype('int16') / 255
    B = np.copy(img_B).astype('int16') / 255
    D = B - A
    D = D + 1
    D = D / 2
    h, w = A.shape
    wm_pts = []
    for y in range(h):
        for x in range(w):
            if D[y, x] == 1:
                wm_pts.append((x, y))
    return wm_pts, D.astype('int16')

In [297]:
wm_pts,wm_mat = getWhiteMinusCnt2(img_A_dither, img_B_dither)

In [298]:
white_mat = img_B.copy()
h,w = white_mat.shape
white_pts = []
for y in range(h):
    for x in range(w):
        if white_mat[y,x] > 238:
            white_mat[y,x] = 1
            white_pts.append((x,y))
        else:
            white_mat[y,x] = 0
print(np.unique(white_mat))

[0 1]


In [299]:
darker_mat = white_mat-wm_mat
darker_mat = darker_mat + 1
darker_mat = (darker_mat/ 2).astype('int16')

In [300]:
# print(np.unique(white_mat-wm_mat))
# print(np.unique(darker_mat))

In [301]:
# print(np.unique(wm_mat))
# vizPtsInArea(img_B.shape,[wm_pts])
# epd.showCv2Img(wm_mat*120)

# epd.showCv2Img(white_mat*120)
# epd.showCv2Img(darker_mat*240)
# epd.showCv2Img(white_mat*200)


## Darker White Minus specially

In [302]:
non_wm_white_pts = list(set(white_pts)-set(wm_pts))

In [303]:
# vizPtsInArea(img_B.shape, [non_wm_white_pts])

In [314]:
def isInImage(x,y,h,w):
    return (x<w and x >= 0 and y < h and y >= 0)
def getDarkerMatAndPts(wmmat, nonwm_white_pts, threshold=4):
    dark_mat = np.zeros_like(wmmat)
    dark_pts = []
    h,w = wmmat.shape
    a = []
    for p in nonwm_white_pts:
        x,y = p
        adjacent_cnt = 0
        for i in [x,x-1,x+1]:
            for j in [y,y-1,y+1]:
                if i==x and j==y:
                    continue
                # print((i,j))
                if isInImage(i,j,h,w):
                    adjacent_cnt = adjacent_cnt + 1
        a.append(adjacent_cnt)
        if adjacent_cnt < threshold:
            dark_mat[y,x] = 1
            dark_pts.append((x,y))
        # print(Counter(a))
    return dark_mat,dark_pts

def getDarkerMatAndPts2(wmmat, nonwm_white_pts, threshold=4):
    dark_mat = np.zeros_like(wmmat)
    dark_pts = []
    h,w = wmmat.shape
    a = []
    for p in nonwm_white_pts:
        x,y = p
        adjacent_cnt = 0
        for i in [x,x-2,x+2]:
            for j in [y,y-2,y+2]:
                if i==x and j==y:
                    continue
                if isInImage(i,j,h,w) and wmmat[j,i] == 1:
                    adjacent_cnt = adjacent_cnt + 1
        # a.append(adjacent_cnt)
        if adjacent_cnt < threshold:
            dark_mat[y,x] = 1
            dark_pts.append((x,y))
        # print(Counter(a))
    return dark_mat,dark_pts

def getDarkerMatAndPts22(wmmat, nonwm_white_pts, threshold=(0,3)):
    dark_mat = np.zeros_like(wmmat)
    dark_pts = []
    h,w = wmmat.shape
    a = []
    for p in nonwm_white_pts:
        x,y = p
        adjacent_cnt = 0
        for i in [x,x-160,x+160]:
            for j in [y,y-160,y+160]:
                if i==x and j==y:
                    continue
                if isInImage(i,j,h,w) and wmmat[j,i] == 1:
                    adjacent_cnt = adjacent_cnt + 1
        # a.append(adjacent_cnt)
        if adjacent_cnt < threshold[1] and adjacent_cnt > threshold[0]:
            dark_mat[y,x] = 1 / adjacent_cnt
            dark_pts.append((x,y))
        # print(Counter(a))
    return dark_mat,dark_pts

In [341]:
# dm, dp = getDarkerMatAndPts(wm_mat, non_wm_white_pts,threshold=9)
# darker_mat2, darker_pts = getDarkerMatAndPts2(wm_mat, non_wm_white_pts,threshold=2)
darker_mat_bd2, darker_pts_bd2 = getDarkerMatAndPts2(wm_mat, non_wm_white_pts,threshold=2)
# darker_mat_bd2, darker_pts_bd2 = getDarkerMatAndPts22(wm_mat, non_wm_white_pts,threshold=1)

# darker_mat2, darker_pts = getDarkerMatAndPts(wm_mat, non_wm_white_pts,threshold=8)


In [342]:
# print(np.unique(dm))
# print(Counter(dm.flatten()))
# print(len(dp))
# epd.showCv2Img(darker_mat2*255)
# print(np.unique(dm))
# print(len(dp))
# vizPtsInArea(img_B.shape, [non_wm_white_pts,wm_pts])

In [343]:
whiteMinusDarker = AfterWhiteEdgeDarker.copy()
# whiteMinusDarker = img_B_Kill_WE.copy()
# img_B_Kill_WE

In [347]:
# path_dst_BD2 = path_out_home + 'B11' + dst_type
whiteMinusDarker = AfterWhiteEdgeDarker.copy()
factor = 18#20
whiteMinusDarker = whiteMinusDarker - darker_mat_bd2*factor
# whiteMinusDarker = whiteMinusDarker + rest_mat*15
whiteMinusDarker = dither.floyd_steinberg(whiteMinusDarker/255)*255
whiteMinusDarker = cv.rotate(whiteMinusDarker, cv.ROTATE_90_CLOCKWISE)
epd.saveAsBlackWhiteImage(path_dst_BD2, whiteMinusDarker)
print('Done')

Done


getDarkerMatAndPts2.threshold=2 factor=13

In [330]:
epd.showCv2Img(darker_mat_bd2*240)
# vizPtsInArea(img_B.shape, [non_wm_white_pts,wm_pts])

# Make BD-2 new

In [132]:
we_cnt_lut, we_pos_lut, we_mat012 = cwe.getWhiteEdgeCnt2(img_A_dither, img_B_dither)
wm_pts,wm_mat = getWhiteMinusCnt2(img_A_dither, img_B_dither)

In [135]:
# epd.showCv2Img(wm_mat*200)

In [145]:
def isInImage(x,y,h,w):
    return (x<w and x >= 0 and y < h and y >= 0)

def getAdjacentMatCnt2(mat,x,y):
    adjacent_cnt = 0
    for i in [x,x-2,x+2]:
        for j in [y,y-2,y+2]:
            if i==x and j==y:
                continue
            if isInImage(i,j,h,w) and mat[j,i] >= 1:
                adjacent_cnt = adjacent_cnt + 1
    return adjacent_cnt
def getAdjacentMatCnt8(mat,x,y):
    adjacent_cnt = 0
    for i in [x,x-8,x+8]:
        for j in [y,y-8,y+8]:
            if i==x and j==y:
                continue
            if isInImage(i,j,h,w) and mat[j,i] >= 1:
                adjacent_cnt = adjacent_cnt + 1
    return adjacent_cnt

In [146]:
andpts = set(we_cnt_lut.keys())&set(wm_pts)
# vizPtsInArea(img_B.shape,[andpts])

In [173]:
img_B = cv.imread(path_src_B, cv.IMREAD_GRAYSCALE)
canvas = img_B.copy()
h,w = canvas.shape
darker_mat_we = np.zeros_like(canvas)
darker_mat_wm = np.zeros_like(canvas)
darker_pts_we = []
darker_pts_wm = []

threshold_we = 6
threshold_wm = 7
for y in range(h):
    for x in range(w):
        we_val = we_mat012[y,x]
        wm_val = wm_mat[y,x]
        if we_val<1 and wm_val<1:
            continue
        if wm_val > 0:
            cnt = getAdjacentMatCnt2(wm_mat,x,y)
            if cnt < threshold_wm:
                darker_mat_wm[y,x] = 1
                darker_pts_wm.append((x,y))
        if we_val > 0:
            cnt = getAdjacentMatCnt8(we_mat012,x,y)
            if cnt < threshold_we:
                darker_mat_we[y,x] = 1
                darker_pts_we.append((x,y))

In [163]:
# epd.showCv2Img(darker_mat_we*240)
# epd.showCv2Img(darker_mat_wm*240)
#

In [190]:
'\\\\192.168.137.114\\myshare\\IT8951\\pic\\exp\\opera\\'
path_dst_BD23 = path_out_home + 'V' + dst_type
factor_wm = 50#20
factor_we = 95#95
img_B = cv.imread(path_src_B, cv.IMREAD_GRAYSCALE)
canvas = img_B.copy()
# canvas = canvas - darker_mat_we*factor_we
canvas = cv.subtract(canvas,darker_mat_wm*factor_wm)
canvas = cv.subtract(canvas,darker_mat_we*factor_we)
# canvas = canvas - darker_mat_wm*factor_wm
canvas = cv.rotate(canvas, cv.ROTATE_90_CLOCKWISE)
epd.saveAsBlackWhiteImage(path_dst_BD23, canvas)
print('Done')

Done


# Visualize All Kinds of Ghosts

## Get Black Edge

In [78]:
def isInImg(img, x, y):
    h, w = img.shape
    return (x in range(w)) and (y in range(h))


def checkAdjacent2(mA, mB, mD, x, y):
    cnt = 0
    if isInImg(mA, x, y):
        d = mD[y, x]
        a = mA[y, x]
        b = mB[y, x]
        cnt = 1 if ((d == 0) and (a == 1)) else 0
    return cnt
def getBlackEdgeCnt2(imgA, imgB):
    A = imgA / 255
    B = imgB / 255
    D = B - A
    BlackEdge = np.zeros_like(D)
    BE_CNT_LUT = {}
    BE_POS_LUT = {}
    h, w = D.shape
    for y in range(h):
        for x in range(w):
            px = D[y, x]
            if px == 1:
                cnt = 0
                pos = 0;
                tmp = checkAdjacent2(A, B, D, x + 1, y)  # right
                cnt = cnt + tmp
                pos = pos | 1 if tmp > 0 else pos
                tmp = checkAdjacent2(A, B, D, x, y + 1)  # down
                cnt = cnt + tmp
                pos = pos | 2 if tmp > 0 else pos
                BlackEdge[y, x] = cnt
                if cnt > 0:
                    BE_CNT_LUT[(x, y)] = cnt
                    BE_POS_LUT[(x, y)] = pos
                pass
            pass
    return BE_CNT_LUT, BE_POS_LUT, BlackEdge

In [79]:
be_cnt_lut, be_pos_lut, be_mat = getBlackEdgeCnt2(img_A_dither,img_B_dither)

In [87]:
%timeit getBlackEdgeCnt2(img_A_dither,img_B_dither)

1.39 s ± 24.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [527]:
print(len(be_cnt_lut))

317213


## 配色方案1

In [81]:
# vizPtsInArea(img_B.shape, [list(we_cnt_lut.keys()), wm_pts])
vizPtsInArea(img_B.shape, [wm_pts,list(be_cnt_lut.keys()),list(we_cnt_lut.keys())], [(0xff,0x7f,0),(0, 0xff, 0x7f),(0x7f,0,0xff)]) # Orange Green Purple
# vizPtsInArea(img_B.shape, [wm_pts], [(0xff,0x7f,0),(0, 0xff, 0x7f),(0x7f,0,0xff)]) # Orange
# vizPtsInArea(img_B.shape, [list(be_cnt_lut.keys())], [(0, 0xff, 0x7f)]) # Green
# vizPtsInArea(img_B.shape, [list(be_cnt_lut.keys())], [(0x7f, 0, 0xff)]) # Orange Green Purple

## 配色方案2

In [86]:
# vizPtsInArea(img_B.shape, [list(we_cnt_lut.keys()), wm_pts])
# vizPtsInArea(img_B.shape, [wm_pts,list(be_cnt_lut.keys()),list(we_cnt_lut.keys())], [(40,120,181),(255, 204, 0),(179,84,69)]) # Orange Green Purple
# vizPtsInArea(img_B.shape, [wm_pts], [(40,120,181)]) # blue
# vizPtsInArea(img_B.shape, [list(be_cnt_lut.keys())], [(255, 204, 0)]) # Yellow
vizPtsInArea(img_B.shape, [list(we_cnt_lut.keys())], [(179,84,69)]) # Red